In [1]:
# !pip -q install pandas neptune transformers matplotlib tqdm scikit-learn scipy opencv-python-headless torch_optimizer torchinfo pyarrow fastai accelerate
# !sudo apt install build-essential --yes

In [1]:
NUM_PROCESSES = 1

In [2]:
from models.model import RNARegModel
from datasets.dataset import RNADataset, collate_fn
from utils.util import seed_everything, clear_everything
from callbacks.callback import SaveLastCallback
from losses.loss import mae_loss, weighted_mae_loss
from metrics.metric import MAE, MAE_ave

from fastai.vision.all import GradientClip, DataLoaders, DataLoader, Metric, Learner, Adam, CSVLogger
from fastai.distributed import *

import pandas as pd
import numpy as np
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, WeightedRandomSampler

from sklearn.model_selection import KFold
from tqdm.autonotebook import tqdm
import gc
import os
from functools import partial

/home/hoyso/miniconda3/envs/hoyso_ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CFG:
    seed = 2023
    n_splits = 5
    fold = 0
    use_all_train = False #use all train data or not, fold will be ignored when True.
    max_len = -1 #input length will never exceed this value, -1 for unbounded. set it to 206 when num_processes>1
    dynamic_len = True #apply dynamic padding on batch or not. If False, input length will be fixed to max_len(max_len must be > 0 for this case). set it to False when num_processes>1
    
    mixed_precision = True

    num_workers = 8
    device = 'cuda'
    # grad_accumulation = 1
    clip_grad = 3.0
    lr = 2e-3 ##for stability. 4e-3 is slightly better in scores.
    weight_decay = 0.01
    epoch = 60 
    warmup = 0.01
    train_batch_size = 256
    valid_batch_size = train_batch_size
    valid_drop_last = False #set it to True when num_processes>1
    train_signal_to_noise_filter = 1.0 #signal_to_noise>this value

    dim = 192
    num_layers = 12
    num_heads = 4
    
    root_dir = '.'
    data_dir = '../datamount'
    output_dir = '../outputs'
    model_dir = 'model_ckpts'
    model_name = 'model'

In [4]:
def train(cfg=CFG):
    os.environ["NCCL_IB_DISABLE"] = "1"
    os.environ["NCCL_P2P_DISABLE"] = "1"
    os.chdir(cfg.root_dir)
    os.makedirs(cfg.output_dir, exist_ok=True)
    os.makedirs(f'{cfg.output_dir}/{cfg.model_dir}', exist_ok=True)
    seed_everything(cfg.seed)
    train_df = pd.read_parquet(f'{cfg.data_dir}/train_data_new.parquet')#.sample(frac=0.01).reset_index(drop=True)

    df = train_df
    if cfg.use_all_train:
        train = df
        valid = df[:0]
    else:
        df['fold'] = -1
        df = df.reset_index(drop=True)
    
        kfold = KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True)
        for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(df)):
            df.loc[valid_idx, 'fold'] = fold_idx
    
        assert not (df['fold']==-1).sum()
        assert len(np.unique(df['fold']))==cfg.n_splits
    
        train = df.loc[df['fold']!=cfg.fold]
        valid = df.loc[df['fold']==cfg.fold]

    train_ds = RNADataset(train, mode='train', SN_filter=False, signal_to_noise_filter=cfg.train_signal_to_noise_filter, dir=cfg.data_dir)
    valid_ds = RNADataset(valid, mode='valid', SN_filter=True, dir=cfg.data_dir)
    train_loader = DataLoader(train_ds,
                        batch_size=cfg.train_batch_size,
                        shuffle=True,
                        sampler=None,
                        num_workers=cfg.num_workers,
                        persistent_workers=True,
                        drop_last=True,
                        pin_memory=True,
                        prefetch_factor=2,
                        create_batch=lambda x: collate_fn(x, cfg.max_len, cfg.dynamic_len),
                       )
    valid_loader = DataLoader(valid_ds,
                        batch_size=cfg.valid_batch_size,
                        shuffle=False,
                        sampler=None,
                        num_workers=cfg.num_workers,
                        persistent_workers=True,
                        drop_last=cfg.valid_drop_last,
                        pin_memory=True,
                        prefetch_factor=2,
                        create_batch=lambda x: collate_fn(x, cfg.max_len, cfg.dynamic_len),
                       )
    model = RNARegModel(dim=cfg.dim, num_layers=cfg.num_layers, num_heads=cfg.num_heads)
    model = torch.compile(model.to(cfg.device))
    data = DataLoaders(train_loader, valid_loader)
    clear_everything()

    learn = Learner(data, 
                model, 
                loss_func=weighted_mae_loss,
                path=cfg.output_dir,
                model_dir=cfg.model_dir,
                cbs=[GradientClip(cfg.clip_grad), SaveLastCallback(cfg.model_name), CSVLogger(fname=f'{cfg.model_name}-log.csv')],
                opt_func=partial(Adam, lr=cfg.lr, mom=0.9, sqr_mom=0.999, eps=1e-8),
            metrics=[MAE_ave()]).to_fp16(enabled=cfg.mixed_precision)

    with learn.distrib_ctx(in_notebook=True, sync_bn=False): #masked sync_bn is not implemented

        learn.fit_one_cycle(cfg.epoch,
                            lr_max=cfg.lr,
                            wd=cfg.weight_decay,
                            pct_start=cfg.warmup)

    return learn

In [5]:
from accelerate.utils import write_basic_config
write_basic_config()
from accelerate import notebook_launcher
notebook_launcher(train, num_processes=NUM_PROCESSES)

Configuration already exists at /home/hoyso/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
Launching training on one GPU.
Training Learner...


epoch,train_loss,valid_loss,mae_ave,time
0,0.199442,0.192536,0.145276,03:44
1,0.185810,0.181758,0.138857,02:19
2,0.181360,0.178604,0.136511,02:19
3,0.177917,0.176190,0.134855,02:19
4,0.176790,0.173685,0.133273,02:20
5,0.173398,0.170592,0.131440,02:19
6,0.171243,0.171730,0.132104,02:19
7,0.169372,0.168189,0.129873,02:20
8,0.168517,0.167031,0.129308,02:19
9,0.167307,0.166713,0.129103,02:19


/home/hoyso/miniconda3/envs/hoyso_ml/lib/python3.9/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/hoyso/miniconda3/envs/hoyso_ml/lib/python3.9/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/hoyso/miniconda3/envs/hoyso_ml/lib/python3.9/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/hoyso/miniconda3/envs/hoyso_ml/lib/python3.9/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
